In [1]:
using Gurobi, JuMP, DataFrames, CSV, Plots

# Utility Function

In [11]:
function distance(x1, y1, x2, y2)
    floatpointcorrection = sin(y1)*sin(y2) + cos(y1)*cos(y2)*cos(x1-x2)
    floatpointcorrection = ifelse(floatpointcorrection > 1, 1.0, floatpointcorrection)
    floatpointcorrection = ifelse(floatpointcorrection < -1, -1.0, floatpointcorrection)
    ß = acos(floatpointcorrection)
    return ß * 6378.1
end;

# Importing data, rough edits very quickly to clean it up

In [3]:
data = CSV.read("./data/final_prod.csv", DataFrame)
select!(data, Not(:Column1))
select!(data, Not(["Area Abbreviation", "Area Code", "Unit", "Item Code"]))
data = data[data[:,7] .!= "Miscellaneous", :]
data = data[data[:,7] .!= "Aquatic Products, Other", :]
population = data[:, [:Area, :population]];
food = data[1, 1:5];

# Partitioning and formatting data for Gurobi

In [4]:
demand = combine(groupby(data[!,1:6], :Area), first)
for row in eachrow(demand)
    row[:fats] = 365 * row[:fats]
    row[:proteins] = 365 * row[:proteins]
    row[:carbs] = 365 * row[:carbs]
    row[:fruits_and_veggies] = 365 * row[:fruits_and_veggies]
end
demand = Matrix(demand[:,2:5]);

In [5]:
loc = unique(data[!,[6,8,9]]);

In [6]:
supply = data[:,[6,7,10]]
supplyT = DataFrame(zeros((length(unique(supply[:,1])), length(unique(supply[:,2])))), :auto)
rename!(supplyT, unique(supply[:,2]));
supplyg = groupby(supply, :Area)
for i in 1:168
    for row in eachrow(supplyg[i])
        supplyT[i, Symbol(row[2])] = row[3]
    end
end

In [12]:
inverse_populations = inv.(unique(population)[!,2])
weighting = CSV.read("./data/weighting.csv", DataFrame)
weighting = Matrix(weighting[:,2:end])
n,p1 = size(supplyT)
n,p2 = size(demand)
lambda = 250*365;
arcDistance = DataFrame(zeros(n, n), :auto)
for i in 1:168
    for j in 1:168
        arcDistance[i, j] = distance(loc[i,2], loc[i,3], loc[j,2], loc[j,3])
    end
end
arcDistance = Matrix(round.(arcDistance, digits=7));

In [14]:
using Random
supply_uncertainty = randn!(MersenneTwister(1234), zeros(50, 168, 74))
supply_uncertainty = round.(abs.(supply_uncertainty .* inv.(maximum(abs.(supply_uncertainty)))) .* 0.6 .+ 0.9, digits=4)
demand_uncertainty = randn!(MersenneTwister(1234), zeros(50, 168, 4))
demand_uncertainty = round.(abs.(demand_uncertainty .* inv.(maximum(abs.(demand_uncertainty)))) .* 0.6 .+ 0.9, digits=4);
uncertainty=50

50

# Optimization

In [15]:
øD = 1 # relative scaling of price per unit when transporting to directly
øH = 0.6 # relative scaling of price per unit when transporting to a central hub as a point of distribution
threshold = 200 # capacity of 10 Boeing 747-400Fs
H = 78.91 # price of per km per metric tonne transport via a Boeing 747-400F
å = 0.55 # relative importance between the two objectives

0.55

## Here I'm trying three different formulations of varying levels of complexity.
The issue is that this doesn't compile locally with the completed (robust and 2nd level network flow implementations) version. 

In [ ]:
model = Model(Gurobi.Optimizer)
set_time_limit_sec(model, 60.0)
@variable(model, X[i=1:n,j=1:n,k=1:p1] .≥ 0)
@variable(model, Y[i=1:n,j=1:n,k=1:p1] .≥ 0)
@variable(model, Z[i=1:n,j=1:n,k=1:p1] .≥ 0)
@variable(model, ∂[i=1:n,j=1:n], binary=true)
@variable(model, ß ≥ 0)
@constraint(model, hubIndicator1[i=1:n], sum(∂[i,:]) .≤ 5)
@constraint(model, hubIndicator2, 5000000 * ∂ .≥ sum(Y[:,:,k] for k in 1:p1))
@constraint(model, realSupply[z=1:uncertainty], sum((X .+ Y)[:,l,:] for l=1:n) .≤ abs.(Matrix(supplyT) .* supply_uncertainty[z,:,:]))
@constraint(model, auxiliary[z=1:uncertainty], ß ≥ inverse_populations' * (Matrix(demand) .* demand_uncertainty[z,:,:] - 10000000 * sum((X+Z)[l,:,:] for l=1:n) * weighting) * inv.([41.3, 107.38, 37.9961, 330.4]))
@constraint(model, hubIndicator, ∂ .≤ inv(threshold) .* sum((Y+Z+X)[:,:,k] for k=1:p1))
@constraint(model,hubEquality[l=1:n,k=1:p1], sum(Y[i,l,k] for i = 1:n) .== sum(Z[l,i,k] for i = 1:n))
@objective(model, Min, (1-å) * H * sum(sum(arcDistance .* sum((øD * X + Y * øH .* ∂ + øD * Z .* ∂)[i,j,:]) for j=1:n) for i=1:n) + (å) * ß * lambda)
optimize!(model)

In [ ]:
model = Model(Gurobi.Optimizer)
set_time_limit_sec(model, 60.0)
@variable(model, X[i=1:n,j=1:n,k=1:p1] .≥ 0)
@variable(model, ß ≥ 0)
@constraint(model, realSupply[z=1:uncertainty], sum(X[:,l,:] for l=1:n) .≤ abs.(Matrix(supplyT) .* supply_uncertainty[z,:,:]))
@constraint(model, auxiliary[z=1:uncertainty], ß ≥ inverse_populations' * (Matrix(demand) .* demand_uncertainty[z,:,:] - 10000000 * sum((X+Z)[l,:,:] for l=1:n) * weighting) * inv.([41.3, 107.38, 37.9961, 330.4]))
@objective(model, Min, (1-å) * H * sum(sum(arcDistance .* sum(X[i,j,:]) for j=1:n) for i=1:n) + (å) * ß * lambda)
optimize!(model)

In [ ]:
model = Model(Gurobi.Optimizer)
set_time_limit_sec(model, 60.0)
@variable(model, X[i=1:40,j=1:40,k=1:74] .≥ 0)
@variable(model, ß ≥ 0, integer=true)
@constraint(model, realSupply, sum(X[:,l,:] for l=1:40) .≤ abs.(Matrix(supplyT)[1:40,:]))
@constraint(model, auxiliary, ß ≥ inverse_populations[1:40]' * (Matrix(demand)[1:40,:] .* 0.7 - sum(X[l,:,:] for l=1:40) * weighting) * inv.([41.3, 107.38, 37.9961, 330.4]))
@objective(model, Min, (1-å) * H * sum(sum(arcDistance[1:40,1:40] .* sum(X[i,j,:]) for j=1:40) for i=1:40) + (å) * ß * lambda)
optimize!(model)

### Analysis

The total distance traveled of all food:

In [46]:
sum(sum(arcDistance[:5,:5] .* sum(value.(X)[i,j,:]) for j=1:5) for i=1:5)

0.0

The total calculated transportation cost of all food items:

In [ ]:
inv(1-å) * H * sum(sum(arcDistance[i,j] * sum(value.(X)[i,j,:] * øD + value.(Y)[i,j,:] * øH * ∂[i,j] + value.(Z)[i,j,:] * øD * ∂[i,j]) for j=1:n) for i=1:n)

The distance traveled of all food going to a distribution hub:

In [ ]:
sum(sum(arcDistance .* sum(value.(Y)[:,:,k] for k=1:p1), dims=1))

The total number of people left hungry over the course of the year; alternatively, the number of meals that aren't provided over the course of the year:

In [ ]:
value.(ß)

The cost incurred by not feeding people based on the parameter set above:

In [ ]:
inv(å) * value.(ß) * lambda

The locations of distribution hubs for America's distribution:

In [ ]:
betas = findall(x -> round(x) != 0, JuMP.value.(∂))